In [1]:
import numpy as np
import cv2
import timeit

In [13]:
def QuantizeColor(img, n=64):
    div = 256//n
    rgb = cv2.split(img)
    q = []
    for ch in rgb:
        vf = np.vectorize(lambda x, div: int(x//div)*div)
        quantized = vf(ch, div)
        q.append(quantized.astype(np.uint8))
    d_img = cv2.merge(q)
    return d_img

def ccv(src, tau=0, n=64):
    img = src.copy()
    row, col, channels = img.shape
    #print col
    if not col == 300:
        aspect = float(300)/float(col)
        #print aspect
        img = cv2.resize(img, None, fx=aspect, fy=aspect, interpolation = cv2.INTER_CUBIC)
    row, col, channels = img.shape
    # blur
    img = cv2.GaussianBlur(img, (3,3),0)
    # quantize color
    img = QuantizeColor(img, n)
    bgr = cv2.split(img)
    if tau == 0:
        tau = row*col * 0.1
    alpha = np.zeros(n)
    beta = np.zeros(n)
    # labeling
    for i,ch in enumerate(bgr):
        ret,th = cv2.threshold(ch,127,255,0)
        ret, labeled, stat, centroids = cv2.connectedComponentsWithStats(th, None, cv2.CC_STAT_AREA, None, connectivity=8)
    # generate ccv
        areas = [[v[4],label_idx] for label_idx,v in enumerate(stat)]
        coord = [[v[0],v[1]] for label_idx,v in enumerate(stat)]
    # Counting
        for a,c in zip(areas,coord):
            area_size = a[0]
            x,y = c[0], c[1]
            if (x < ch.shape[1]) and (y < ch.shape[0]):
                bin_idx = int(float(ch[y,x])/float(256/float(n)))
                if area_size >= tau:
                    alpha[bin_idx] = alpha[bin_idx] + area_size
                else:
                    beta[bin_idx] = beta[bin_idx] + area_size
    return alpha, beta



def extract_feature(img):
    frame = img
    roiWid = 10
    #roiEdg = 8
    roiEdg = 10
    src_height, src_width, src_channels = frame.shape
    roiX = int(src_width / roiWid)
    roiWidth = roiX * roiEdg
    roiY = int(src_height / roiWid)
    roiHeight = roiY * roiEdg
    RoiImg = frame[roiY : roiY+roiHeight, roiX : roiX+roiWidth]
    
#     n=128
#     alpha, beta = ccv(RoiImg, tau=0,n=n)
#     CCV = alpha.tolist()+beta.tolist()
#     X = [x for x in range(n*2)]
#     Y = alpha.tolist()+beta.tolist()
    
    
    HSV = cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
    H,S,V = cv2.split(HSV)
    Vue_STD = np.std(V)
    Gimg = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    G_STD = np.std(Gimg)
    hist,bins = np.histogram(img.flatten(),256,[0,256])
    #hist2 = cv2.calcHist([img],[0],None,[256],[0,256])
    #hist_STD = np.std(hist)
    #hist_Mean = np.mean(hist)
    r = img[...,2]
    g = img[...,1]
    b = img[...,0]
    rY = 0.212655;
    gY = 0.715158;
    bY = 0.072187;
    inner = RoiImg
    ir = inner[...,2]
    ig = inner[...,1]
    ib = inner[...,0]

    Luminance = r*0.2126+g*0.7152+0.0722*b
    Li = ir*0.2126+ig*0.7152+0.0722*ib
    hei,wid,chan = img.shape
    ih,iw,ic = inner.shape
    Lum_value = (np.sum(Luminance)-np.sum(Li))/((hei*wid)-(ih*iw))
    
    mr = np.sqrt((np.mean(r)-119)*(np.mean(r)-119))
    mg = np.sqrt((np.mean(g)-119)*(np.mean(g)-119))
    mb = np.sqrt((np.mean(b)-119)*(np.mean(b)-119))
    GW_value = (mr+mg+mb)/3
    
    #return (Vue_STD,Lum_value,G_STD,GW_value,np.median(hist),Y)
    #return Y
    return (Vue_STD,Lum_value,G_STD,GW_value,np.median(hist))

In [14]:
img_src = '/root/ND_classifier/Video_Dark/LIVE/LIVE_CAMERA_16pm.mp4_frame/13.jpg'
img_src = '/root/ND_classifier/Face_data/1_Face_White/13.jpg'
img_src = '/root/ND_classifier/Video_Dark/LIVE/LIVE_CAMERA_16pm.mp4_frame/frame13.jpg'

img = cv2.imread(img_src)
%timeit out = extract_feature(img)

10 loops, best of 3: 156 ms per loop


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
#fig = plt.figure()
#ax = fig.add_subplot(111)
gyimg = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
plt.subplot(221),plt.plot(out[7])
plt.subplot(222),plt.hist(img.ravel(),256,[0,256])
plt.subplot(223),plt.imshow(img)
plt.subplot(224),plt.imshow(gyimg,'gray')
plt.show()

In [ ]:
plt.subplot(111),plt.imshow(img)

In [ ]:
def QuantizeColor(img, n=64):
    div = 256//n
    rgb = cv2.split(img)
    q = []
    for ch in rgb:
        vf = np.vectorize(lambda x, div: int(x//div)*div)
        quantized = vf(ch, div)
        q.append(quantized.astype(np.uint8))
    d_img = cv2.merge(q)
    return d_img

def ccv(src, tau=0, n=64):
    img = src.copy()
    row, col, channels = img.shape
    #print col
    if not col == 300:
        aspect = float(300)/float(col)
        #print aspect
        img = cv2.resize(img, None, fx=aspect, fy=aspect, interpolation = cv2.INTER_CUBIC)
    row, col, channels = img.shape
    # blur
    img = cv2.GaussianBlur(img, (3,3),0)
    # quantize color
    img = QuantizeColor(img, n)
    bgr = cv2.split(img)
    if tau == 0:
        tau = row*col * 0.1
    alpha = np.zeros(n)
    beta = np.zeros(n)
    # labeling
    for i,ch in enumerate(bgr):
        ret,th = cv2.threshold(ch,127,255,0)
        ret, labeled, stat, centroids = cv2.connectedComponentsWithStats(th, None, cv2.CC_STAT_AREA, None, connectivity=8)
    # generate ccv
        areas = [[v[4],label_idx] for label_idx,v in enumerate(stat)]
        coord = [[v[0],v[1]] for label_idx,v in enumerate(stat)]
    # Counting
        for a,c in zip(areas,coord):
            area_size = a[0]
            x,y = c[0], c[1]
            if (x < ch.shape[1]) and (y < ch.shape[0]):
                bin_idx = int(float(ch[y,x])/float(256/float(n)))
                if area_size >= tau:
                    alpha[bin_idx] = alpha[bin_idx] + area_size
                else:
                    beta[bin_idx] = beta[bin_idx] + area_size
    return alpha, beta


In [ ]:
import matplotlib.pyplot as plt
img_src = '/root/ND_classifier/Face_data/1_Face_White/13.jpg'
img = cv2.imread(img_src)
n=128
alpha, beta = ccv(img, tau=0,n=n)
CCV = alpha.tolist()+beta.tolist()
X = [x for x in range(n*2)]
Y = alpha.tolist()+beta.tolist()
im = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.close()
plt.subplot(2,1,1)
plt.imshow(im)
plt.subplot(2,1,2)
plt.bar(X, Y, align='center')
#plt.yscale('log')
#plt.xticks(X, (['alpha']*n)+(['beta']*n))
plt.show() 
print Y

In [ ]:
img_src = '/root/ND_classifier/Video_Dark/LIVE/LIVE_CAMERA_16pm.mp4_frame/frame13.jpg'
img = cv2.imread(img_src)
#print img
n=128
alpha, beta = ccv(img, tau=0,n=n)
CCV = alpha.tolist()+beta.tolist()
X = [x for x in range(n*2)]
Y = alpha.tolist()+beta.tolist()
im = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
print alpha,beta
print Y
plt.close()
plt.subplot(2,1,1)
plt.imshow(im)
plt.subplot(2,1,2)
plt.bar(X, Y, align='center')
#plt.yscale('log')
#plt.xticks(X, (['alpha']*n)+(['beta']*n))
plt.show() 

In [ ]:
import skimage

In [ ]:
def normalize(img, level_min=1, level_max=256, threshold=None):

    tmp_img = np.array(img)
    if threshold is None:
        threshold = tmp_img.min()
    tmp_img[tmp_img<threshold] = -1
    assert level_min < level_max, "level_min must be smaller than level_max"
    slope = (level_max - level_min) / (img.max() - threshold)
    intercept = - threshold * slope
    tmp_img = tmp_img * slope + intercept + level_min
    

class GLCM:
    def __init__(self, img, theta=[0, 1], level_min=1, level_max=256, threshold=None):


        assert len(img.shape) == 2, 'image must be 2D'
        self.img, self.slope, self.intercept = \
            normalize(img, level_min, level_max, threshold)
        self.n_level = (level_max - level_min) + 1
        self.level_min = level_min
        self.level_max = level_max
        self.theta = theta
        self.matrix = self._construct_matrix()
        self.features = self._calc_features()

    def _calc_features(self):


        I, J = np.ogrid[self.level_min:self.level_max+1,
                        self.level_min:self.level_max+1]
        mat = np.array(self.matrix)
        features = {}
        features['uniformity'] = (mat**2).sum()
        features['entropy'] = -(mat[mat>0]*np.log(mat[mat>0])).sum()
        features['dissimilarity'] = (mat*np.abs(I-J)).sum()
        features['contrast'] = (mat*((I-J)**2)).sum()
        features['homogeneity'] = (mat/(1+np.abs(I-J))).sum()
        features['inverse difference moment'] = (mat/(1+(I-J)**2)).sum()
        features['maximum probability'] = mat.max()*100
        return features


    def _construct_matrix(self):

        mat = np.zeros((self.n_level, self.n_level)).astype(np.float)
        unique = np.unique(self.img)
        width = self.img.shape[1]
        height = self.img.shape[0]
        print(self.img.max())
        print(self.img.min())
        for uni in unique:
            if uni < self.level_min:
                continue
            indices = np.argwhere(self.img == uni)
            for idx in indices:
                pos = np.array(idx + self.theta, dtype=np.int32)
                if 0 <= pos[0] < height and 0 <= pos[1] < width:
                    neighbor_value = self.img[pos[0], pos[1]]
                    if neighbor_value >= self.level_min:
                        mat[self.img[idx[0], idx[1]]-self.level_min,
                            self.img[pos[0], pos[1]]-self.level_min] += 1
                pos = idx + (self.theta*-1)
                if 0 <= pos[0] < height and 0 <= pos[1] < width:
                    neighbor_value = self.img[pos[0], pos[1]]
                    if neighbor_value >= self.level_min:
                        mat[self.img[idx[0], idx[1]]-self.level_min,
                            self.img[pos[0], pos[1]]-self.level_min] += 1
        return mat